This is the main notebook of the report, it'll contain all of our findings and developments.

In [1]:
import DataReader
import Word2VecAverage

x_test_set, x_train_set, y_test_set, y_train_set = DataReader.split_sheet_into_test_training_per_word()
w2v = Word2VecAverage.Word2VecAverage()
w2v.train_word2vec()

x_test_features = w2v.create_feature_set(x_test_set)

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
clf = LogisticRegression()
clf.fit(x_test_features, y_test_set)
x_train_features = w2v.create_feature_set(x_train_set)
test_pred_clf = clf.predict(x_test_features)
metrics.accuracy_score(test_pred_clf, y_test_set)

C:\Users\lolco\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\lolco\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6946293143476242

# Ngram

In [3]:
import Ngram
from DataReader import split_sheet_into_test_training_per_word
x_test, x_train, y_test, y_train = split_sheet_into_test_training_per_word()
n=2

One model:

In [4]:
x_clean = Ngram.process_1(x_train, y_train, n)
model = Ngram.get_model( x_clean,n)
print('correct, error, doesn\'t know (in%)')
print(Ngram.compute_score(x_test, y_test, model, n))

correct, error, doesn't know (in%)
(0.6494350719702833, 0.03993189908682866, 0.3106330289428881)


Multiple models (one per definition):

In [5]:
x1,x2,x3,x4 = Ngram.process_4(x_train, y_train, n)
models = []
xs = [x1,x2,x3,x4]
for x in xs:
    models.append(Ngram.get_model(x, n))
print('correct, error, doesn\'t know (in%)')
print(Ngram.compute_score(x_test, y_test, models, n))

correct, error, doesn't know (in%)
(0.6404581334158799, 0.04890883764123201, 0.3106330289428881)


# BoW approach

In [6]:
import Bow
mod = Bow.Bow(x_train, y_train)
print("correct, wrong, noidea")
print(mod.get_score(x_test,y_test))

correct, wrong, noidea
(0.8336170871382139, 0.09116235876799257, 0.07522055409379352)


# Baseline

In [10]:
from MostCommon import MostCommon
from sklearn import metrics
import numpy as np
mc = MostCommon()
counts = mc.createCounts()
predicted = mc.classify(mc.x_test, counts)
testl = np.ravel(mc.y_test)
print(metrics.classification_report(testl, predicted))

              precision    recall  f1-score   support

           -       0.00      0.00      0.00        84
           1       0.94      0.11      0.20      4342
           2       0.33      0.92      0.48      1832
           3       0.24      0.99      0.39       168
           4       0.33      1.00      0.50        35

    accuracy                           0.37      6461
   macro avg       0.37      0.60      0.31      6461
weighted avg       0.73      0.37      0.28      6461



C:\Users\lolco\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Semantic dependencies

In [8]:
from SemanticDep import SemanticDep
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
sd = SemanticDep()
f = sd.createFeatures(sd.x_train)
l = np.ravel(sd.y_train)
testf = sd.createFeatures(sd.x_test)
testl = np.ravel(sd.y_test)

# train the model on the features
clf = MultinomialNB()
clf.fit(f, l)

# predict the labels for the test data
predicted = clf.predict(testf)
# output your results
print(metrics.classification_report(testl, predicted))

              precision    recall  f1-score   support

           -       0.14      0.79      0.23        85
           1       0.88      0.69      0.78      4347
           2       0.62      0.69      0.65      1825
           3       0.31      0.81      0.45       170
           4       0.29      0.79      0.43        34

    accuracy                           0.70      6461
   macro avg       0.45      0.75      0.51      6461
weighted avg       0.78      0.70      0.72      6461

              precision    recall  f1-score   support

           -       0.14      0.85      0.25        87
           1       0.87      0.70      0.78      4361
           2       0.60      0.63      0.62      1829
           3       0.30      0.76      0.43       157
           4       0.18      0.93      0.30        27

    accuracy                           0.69      6461
   macro avg       0.42      0.77      0.47      6461
weighted avg       0.77      0.69      0.72      6461

